<a href="https://colab.research.google.com/github/jchenRCL/jchenRCL/blob/master/Visualizing_funding_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualizing a funding pipeline using the Dimensions API


In [ ]:
endpoint = "https://app.dimensions.ai"

!pip install dimcli -U --quiet

# import all libraries and login
import pandas as pd
import numpy as np
import dimcli
from getpass import getpass
import json

import datetime

print("enter you username")
username = input()
print("enter your password")
password = getpass()
dimcli.login(username, password)
dsl = dimcli.Dsl()

In [ ]:
import datetime
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
#"grid.214458.e"

In [ ]:
%%dslloopdf

search grants 
    where research_orgs.id = "grid.1008.9" 
  
return grants[all] 

In [ ]:
dsl.query_iterative("""search grants 
    where research_orgs.id = "grid.1008.9" 
return grants[all]""").as_dataframe()


In [ ]:
dsl_last_results[['active_year','start_date']]

## Make a function to calculate awarded funding by active year

In [ ]:
def get_award_fraction(id,start_date, end_date, active_year):

  #print(start_date, end_date, active_year)

  #If there is a start date but no end date, assume all funding is in the year awarded
  if (type(start_date) == str) and (type(end_date) != str):
    s_date = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    duration_m = 12

  #For awards with both a start date and an end date, calculate a duration
  elif (type(start_date) == str) and (type(end_date) == str):
    s_date = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    e_date = datetime.datetime.strptime(end_date, "%Y-%m-%d")
    duration_m = ((e_date.year - s_date.year) * 12) + (e_date.month - s_date.month) 
  
  # Otherwise there is nothing we can do with this grant - tell me about it
  # Assume start and end dates are on the last day of the month

  else:
    print(id,start_date, end_date, active_year)
    return 0
  
  if duration_m > 12:
    if s_date.year == active_year:
      return (12 - s_date.month)/duration_m
    if e_date.year == active_year:
      return (e_date.month)/duration_m
    else:
      return 12/duration_m
  else:
    return 1




## Take the list of active years, and make a row for each active year

explode!  



In [ ]:
funding = dsl_last_results.explode('active_year') # explode made each active year in each row 
funding[['active_year']]


Now we can use our function to calculate the amount of funding that can be added proportionally to the year from the amount awarded. (choosing usd for this example...)

In [ ]:
# calculate the award fraction for each active year
funding['award_fraction'] = funding.apply(lambda f: f.funding_usd * get_award_fraction(f.id,f.start_date, f.end_date, f.active_year), axis=1)

## Plot the awarded funding by active year

In [ ]:
# make sure all year values are valid. (get rid of rows that don't have a start year)
funding['start_year'] = funding['start_year'].fillna(0).astype(int)

ay = funding.groupby(['active_year']).\
    agg({'award_fraction':'sum'}).\
    reset_index()

In [ ]:
sns.set_palette("Set1")
ay[ay['active_year'].isin(range(2011,2025))].plot(x='active_year', y='award_fraction')

## What Story does this graph tell?

Where did the funding come from? (which start year?)

In [ ]:
# we are only interested in looking at awards since 2009 in detail
funding.start_year = funding.start_year.apply(lambda y: y if y > 2009 else 2009)

# add up the award fractions by actvie year
fp = funding.groupby(['start_year','active_year']).\
    agg({'award_fraction':'sum'}).\
    reset_index().\
    pivot(columns='start_year', index='active_year', values='award_fraction')


# now we have a matrix of funding by active year, broken down by the year that it was awarded
fp[fp.index.isin(range(2011,2025,1))][range(2009,2021,1)]

In [ ]:
# Set up the graph, choose some colours and figure size
sns.set_palette(["lightgrey"]+sns.color_palette("cubehelix_r", 11))
fig, ax = plt.subplots(1, figsize=(15,10))

fp[fp.index.isin(range(2011,2030,1))][range(2009,2021,1)].\
    rename(columns={2009:'All years. to 2009'}).\
    plot(kind='bar', 
         stacked=True,
         title="Funding pipeline", 
         ax=ax)

# Format the legend
ax.legend(bbox_to_anchor=(0., -.25, 1., .102), loc='lower left',
           ncol=4, 
          mode="expand", 
          borderaxespad=0., 
          facecolor='white',
          title="year of award (start_year)")

# Format y axis
ax.get_yaxis().set_major_formatter(
    matplotlib.ticker.FuncFormatter(lambda y, p: format(int(y/1000000), ',')))
ax.set_ylabel("Awarded Funding in Millions")

ax.set_xlabel("Funding by active year")

In [ ]:

fop_order = list(funding.groupby(['funding_org_name']).\
    agg({'award_fraction':'sum'}).\
    reset_index().\
    sort_values(by='award_fraction', ascending=False).funding_org_name)

funding.funding_org_name = funding.funding_org_name.apply(lambda o: o if o in fop_order[:20] else 'Other Funding Body')

sns.set_palette( 
    
                 sns.color_palette("Greens", 5) 
               + sns.color_palette("Blues_r", 5) 
               + sns.color_palette("Reds", 10)

               )

fig, ax = plt.subplots(1, figsize=(15,10))


fop = funding.groupby(['funding_org_name','active_year']).\
    agg({'award_fraction':'sum'}).\
    reset_index().\
    pivot(columns='funding_org_name', index='active_year', values='award_fraction')[fop_order[:20]]

fop[fop.index.isin(range(2011,2030,1))].\
    plot(kind='bar', 
         stacked=True, 
         ax=ax,
         title="Funding pipline by sponsor")

# Format legend
ax.legend(bbox_to_anchor=(0., -.45, 1., .102), loc='lower left',
           ncol=2, 
          mode="expand", 
          borderaxespad=0., 
          facecolor='white',
          title="Sponsor name")

# Format axis
ax.get_yaxis().set_major_formatter(
    matplotlib.ticker.FuncFormatter(lambda y, p: format(int(y/1000000), ',')))
ax.set_ylabel("Awarded funding in Millions")

ax.set_xlabel("Funding by active award year")

In [ ]:
funding['total FOR_first'] = funding.FOR_first.apply(lambda l: len(l) if type(l) == list else 1)
FOR_funding = funding.explode('FOR_first')
FOR_funding['FOR name'] = FOR_funding.FOR_first.apply(lambda d: d['name'] if type(d) == dict else "Unclassified")
FOR_funding['award_fraction'] = FOR_funding['award_fraction'] / FOR_funding['total FOR_first']

funding = FOR_funding

sns.set_palette( 
    
                  sns.color_palette("Greens", 8)[0:5] 
                + sns.color_palette("Blues", 4)[0:2] 
                + sns.color_palette("Greens", 8)[5:8]
                + sns.color_palette("Blues", 4)[2:3]
                + sns.color_palette("Reds", 10)[0:5]
                + sns.color_palette("Blues", 4)[3:4]
                + sns.color_palette("Reds", 5)[5:10]

               )

fig, ax = plt.subplots(1, figsize=(15,10))

#funding.start_year = funding.start_year.apply(lambda y: y if y > 2009 else 2009)

print(funding.start_year.unique())

fp = funding.groupby(['FOR name','active_year']).\
    agg({'award_fraction':'sum'}).\
    reset_index().\
    pivot(columns='FOR name', index='active_year', values='award_fraction')

fp[fp.index.isin(range(2011,2025,1))].\
    plot(kind='bar', stacked=True, ax=ax, title='Funding piplie for COVID-19 authors. Funding tagged by the Field of Research of award ')

ax.legend(bbox_to_anchor=(0., -.55, 1., .102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0., facecolor='white', title="Field of Research")

# Format y axis
ax.get_yaxis().set_major_formatter(
    matplotlib.ticker.FuncFormatter(lambda y, p: format(int(y/1000000), ',')))
ax.set_ylabel("Awarded funding in Millions")

ax.set_xlabel("Funding by active year")

## COVID-19 Example

What does the funding pipeline look like for researchers who are contributing to COVID-19 research?

Recepie:

1. Get the grants

  1.1 get the covid papers. 

  1.2 find out which grants they are acknowledging. 

  1.3 get the details of all active grants

  1.4 add any grants that actively reference covid-19

2. Analyse the pipeline as before
  



In [ ]:
# Step 1 - get the COVID-19 papers

numdays = (datetime.datetime.today() - datetime.datetime(2020, 1, 1, 0, 0)).days
print(numdays)
base = datetime.datetime.today()
datelist = sorted([(base - datetime.timedelta(days=x)).strftime('%Y-%m-%d')
             for x in range(0,numdays+7,7)])
datelist

dfl = []
for dl in [ datelist[i:i+2] for i, d in enumerate(datelist)]:
    print (len(dl))
    if len(dl)==2:
        dfl.append(
            dsl.query_iterative(
                f"""
                         search publications in full_data for "   \\"2019-nCoV\\"
                                                    OR \\"COVID-19\\" 
                                                    OR \\"SARS-CoV-2\\" 
                                                    OR \\"HCoV-2019\\" 
                                                    OR \\"hcov\\" 
                                                    OR \\"NCOVID-19\\" 
                                                    OR \\"severe acute respiratory syndrome coronavirus 2\\" 
                                                    OR \\"severe acute respiratory syndrome corona virus 2\\" 
                                                    OR ((\\"coronavirus\\"  OR \\"corona virus\\") AND (Wuhan OR China OR novel))"
                          where year=2020
                          and date_inserted > "{dl[0]}" 
                          and date_inserted <= "{dl[1]}" 
                          return publications[id+authors+supporting_grant_ids]
                """
            )
        )




In [ ]:
pubs = pd.concat([res.as_dataframe() for res in dfl])
authors = pd.concat([res.as_dataframe_authors() for res in dfl]) 
authors.to_pickle('covid_authos.pkl')   
pubs.to_pickle('covid_publications.pkl')

In [ ]:


pubs = pd.read_pickle('covid_publications.pkl')

# Step 2 - get the grant details

grant_ids = list(pubs.explode('supporting_grant_ids').supporting_grant_ids.dropna().unique()
)

print(len([grant_ids[n:n+250] for n in range(0,len(grant_ids),250)])) # 250 is the interval 

query_results = []

for gids in [grant_ids[n:n+250] for n in range(0,len(grant_ids),250)]:

      mygrantslist = json.dumps(gids)

      query_results.append(
          
                  dsl.query_iterative(f"""
                        search grants
                            where id in {mygrantslist}
                            and active_year=2020
                            return grants[all]
                        """).as_dataframe()
      )

# also add direct covid-19 grants (there are not very many yet, but this will change rapidly)

query_results.append(
          
                  dsl.query_iterative(f"""
                        search grants for {covidsearch}
                            where 
                                start_year=2020
                            return grants[all]
                        """).as_dataframe()
      )



# Step 3 -dedupulicate grants
covid_funding_grants = pd.concat(query_results).\
                drop_duplicates(subset='id')




In [ ]:

funding = covid_funding_grants.explode('active_year')

funding['award_fraction'] = funding.apply(lambda f: f.funding_usd * get_award_fraction(f.id,f.start_date, f.end_date, f.active_year), axis=1)
funding['start_year'] = funding['start_year'].fillna(0).astype(int)

fop_order = list(funding.groupby(['funding_org_name']).\
    agg({'award_fraction':'sum'}).\
    reset_index().\
    sort_values(by='award_fraction', ascending=False).funding_org_name)

sns.set_palette( sns.color_palette("Greens", 5)+sns.color_palette("Blues_r", 5)+sns.color_palette("Reds", 10))
fig, ax = plt.subplots(1, figsize=(15,10))

#funding.start_year = funding.start_year.apply(lambda y: y if y > 2009 else 2009)

funding.funding_org_name = funding.funding_org_name.apply(lambda o: o if o in fop_order[:20] else 'Other Funding Body')

print(funding.start_year.unique())

fop = funding[['funding_org_name','active_year','award_fraction']].\
    groupby(['funding_org_name','active_year']).\
    agg({'award_fraction':'sum'}).\
    reset_index().\
    pivot(columns='funding_org_name', index='active_year', values='award_fraction')[funding.funding_org_name.unique()]

fop[fop.index.isin(range(2011,2030,1))].\
    plot(kind='bar', stacked=True, ax=ax)

ax.legend(bbox_to_anchor=(0., -.4, 1., .102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0., facecolor='white')

In [ ]:
sns.set_palette(["lightgrey"]+sns.color_palette("cubehelix_r", 10)+["red"])
fig, ax = plt.subplots(1, figsize=(15,10))

funding.start_year = funding.start_year.apply(lambda y: y if y > 2009 else 2009)

print(funding.start_year.unique())

fp = funding.groupby(['start_year','active_year']).\
    agg({'award_fraction':'sum'}).\
    reset_index().\
    pivot(columns='start_year', index='active_year', values='award_fraction')

fp[fp.index.isin(range(2011,2030,1))][range(2009,2021,1)].\
    rename(columns={2009:'-2009'}).\
    plot(kind='bar', stacked=True, ax=ax)

ax.legend(bbox_to_anchor=(0., -.25, 1., .102), loc='lower left',
           ncol=4, mode="expand", borderaxespad=0., facecolor='white',
          title="award start year")

# Format y axis
ax.get_yaxis().set_major_formatter(
    matplotlib.ticker.FuncFormatter(lambda y, p: format(int(y/1000000), ',')))
ax.set_ylabel("Awarded Funding in Millions")

ax.set_xlabel("Funding by active year")

## COVID -19 researchers -> active grants

Look at all researchers who have published a covid-19 paper, and look at their funding pipeline

In [ ]:

authors = pd.read_pickle('covid_authos.pkl')   

# Step 2 - get the grant details

researcher_ids = list(authors[authors['researcher_id'] != ''].researcher_id.unique())



query_results = []

for gids in [researcher_ids[n:n+250] for n in range(0,len(researcher_ids),250)]:

      myreslist = json.dumps(gids)

      query_results.append(
          
                  dsl.query_iterative(f"""
                        search grants
                            where researchers.id in {myreslist}
                            and active_year=2020
                            return grants[all]
                        """).as_dataframe()
      )

# also add direct covid-19 grants

query_results.append(
          
                  dsl.query_iterative(f"""
                        search grants for {covidsearch}
                            where 
                                start_year=2020
                            return grants[all]
                        """).as_dataframe()
      )



# Step 3 -depulicate grants
covid_funding_from_researchers = pd.concat(query_results).\
                drop_duplicates(subset='id')

In [ ]:
funding = covid_funding_from_researchers.explode('active_year')

funding['award_fraction'] = funding.apply(lambda f: f.funding_usd * get_award_fraction(f.id,f.start_date, f.end_date, f.active_year), axis=1)
funding['start_year'] = funding['start_year'].fillna(0).astype(int)

fop_order = list(funding.groupby(['funding_org_name']).\
    agg({'award_fraction':'sum'}).\
    reset_index().\
    sort_values(by='award_fraction', ascending=False).funding_org_name)

sns.set_palette( sns.color_palette("Greens", 6)+sns.color_palette("Blues_r", 5)+sns.color_palette("Reds", 10))
fig, ax = plt.subplots(1, figsize=(15,10))


funding.funding_org_name = funding.funding_org_name.apply(lambda o: o if o in fop_order[:20] else 'Other Funding Body')

fop = funding[['funding_org_name','active_year','award_fraction']].\
    groupby(['funding_org_name','active_year']).\
    agg({'award_fraction':'sum'}).\
    reset_index().\
    pivot(columns='funding_org_name', index='active_year', values='award_fraction')[funding.funding_org_name.unique()]

fop[fop.index.isin(range(2011,2030,1))].\
    plot(kind='bar', stacked=True, ax=ax, title="Funding pipline for active research associated with authors of COVID-19 papers")




ax.legend(bbox_to_anchor=(0., -.50, 1., .102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0., facecolor='white', title="Funding by award sponsor")

# Format axis
ax.get_yaxis().set_major_formatter(
    matplotlib.ticker.FuncFormatter(lambda y, p: format(int(y/1000000), ',')))
ax.set_ylabel("Awarded funding in Millions")

ax.set_xlabel("Funding by active award year")

In [ ]:
sns.set_palette(["lightgrey"]+sns.color_palette("cubehelix_r", 10)+["red"])
fig, ax = plt.subplots(1, figsize=(15,10))

funding.start_year = funding.start_year.apply(lambda y: y if y > 2009 else 2009)

print(funding.start_year.unique())

fp = funding.groupby(['start_year','active_year']).\
    agg({'award_fraction':'sum'}).\
    reset_index().\
    pivot(columns='start_year', index='active_year', values='award_fraction')

fp[fp.index.isin(range(2011,2025,1))][range(2009,2021,1)].\
    rename(columns={2009:'-2009'}).\
    plot(kind='bar', stacked=True, ax=ax, title='Funding piplie for COVID-19 authors. Funding tagged by the start year of award ')

ax.legend(bbox_to_anchor=(0., -.25, 1., .102), loc='lower left',
           ncol=4, mode="expand", borderaxespad=0., facecolor='white', title="Funding by start year")

# Format y axis
ax.get_yaxis().set_major_formatter(
    matplotlib.ticker.FuncFormatter(lambda y, p: format(int(y/1000000), ',')))
ax.set_ylabel("Awarded funding in Millions")

ax.set_xlabel("Funding by active year")

## COVID funded pipeline by field of research

In [ ]:
funding['total FOR_first'] = funding.FOR_first.apply(lambda l: len(l) if type(l) == list else 1)
FOR_funding = funding.explode('FOR_first')
FOR_funding['FOR name'] = FOR_funding.FOR_first.apply(lambda d: d['name'] if type(d) == dict else "Unclassified")
FOR_funding['award_fraction'] = FOR_funding['award_fraction'] / FOR_funding['total FOR_first']

FOR_funding

sns.set_palette(["lightgrey"]+sns.color_palette("cubehelix_r", 21)+["red"])
fig, ax = plt.subplots(1, figsize=(15,10))

#funding.start_year = funding.start_year.apply(lambda y: y if y > 2009 else 2009)

print(FOR_funding.start_year.unique())

fp = FOR_funding.groupby(['FOR name','active_year']).\
    agg({'award_fraction':'sum'}).\
    reset_index().\
    pivot(columns='FOR name', index='active_year', values='award_fraction')

fp[fp.index.isin(range(2011,2025,1))].\
    plot(kind='bar', stacked=True, ax=ax, title='Funding piplie for COVID-19 authors. Funding tagged by the Field of Research of award ')

ax.legend(bbox_to_anchor=(0., -.55, 1., .102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0., facecolor='white', title="Field of Research")

# Format y axis
ax.get_yaxis().set_major_formatter(
    matplotlib.ticker.FuncFormatter(lambda y, p: format(int(y/1000000), ',')))
ax.set_ylabel("Awarded funding in Millions")

ax.set_xlabel("Funding by active year")

In [ ]:
RCDC_funding = funding.explode('category_for')

In [ ]:
funding.columns

In [ ]:
funding['total RCDC'] = funding.category_rcdc.apply(lambda l: len(l) if type(l) == list else 1)
RCDC_funding = funding.explode('category_rcdc')
RCDC_funding['RCDC name'] = RCDC_funding.category_rcdc.apply(lambda d: d['name'] if type(d) == dict else "Unclassified")
RCDC_funding['award_fraction'] = RCDC_funding['award_fraction'] / RCDC_funding['total RCDC']

RCDC_funding

sns.set_palette(["lightgrey"]+sns.color_palette("cubehelix_r", 21)+["red"])
fig, ax = plt.subplots(1, figsize=(15,10))

#funding.start_year = funding.start_year.apply(lambda y: y if y > 2009 else 2009)

print(RCDC_funding.start_year.unique())

fp = RCDC_funding.groupby(['RCDC name','active_year']).\
    agg({'award_fraction':'sum'}).\
    reset_index().\
    pivot(columns='RCDC name', index='active_year', values='award_fraction')

top_20 = fp[fp.index == 2020].transpose().sort_values(by=2020, ascending=False).head(20).index.to_list()

fp[fp.index.isin(range(2011,2025,1))][top_20].\
    plot(kind='bar', stacked=True, ax=ax, title='Funding piplie for COVID-19 authors. Funding tagged by RCDC')

ax.legend(bbox_to_anchor=(0., -.55, 1., .102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0., facecolor='white', title="RCDC Category")

# Format y axis
ax.get_yaxis().set_major_formatter(
    matplotlib.ticker.FuncFormatter(lambda y, p: format(int(y/1000000), ',')))
ax.set_ylabel("Awarded funding in Millions")

ax.set_xlabel("Funding by active year")

In [ ]:
fp[fp.index == 2020].transpose().sort_values(by=2020, ascending=False).head(20).index.to_list()